In [1]:
import os
import shutil

import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import PPO

# Initialize video recording output directory
output_folder = os.path.join(os.getcwd(), 'gym-results')
os.makedirs(output_folder, exist_ok=True)

train_output_folder = os.path.join(output_folder, 'train')
shutil.rmtree(train_output_folder, ignore_errors=True)
os.makedirs(train_output_folder, exist_ok=True)

# Environment
env = RecordVideo(
    gym.make("Acrobot-v1", max_episode_steps=512, render_mode="rgb_array"),
    train_output_folder,
    episode_trigger=(lambda episode_id: True),
)
# Train a policy
model = PPO("MlpPolicy", env, verbose=1, seed=0).learn(total_timesteps=10000)
env.close()

2023-12-08 20:56:31.289665: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jsukpk/Workstation/CSEM/cs395t-final-project/.env/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-0.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-1.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-2.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-2.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-3.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-3.mp4



...

Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-16.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-17.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-17.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-17.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-18.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-18.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-18.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-19.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-19.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-19.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 506         |
|    ep_rew_mean          | -506        |
| time/                   |             |
|    fps                  | 112         |
|    iterations           | 5           |
|    time_elapsed         | 91          |
|    total_timesteps      | 10240       |
| train/                  |             |
|    approx_kl            | 0.009465012 |
|    clip_fraction        | 0.0242      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.00254    |
|    learning_rate        | 0.0003      |
|    loss                 | 15.7        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.00291    |
|    value_loss           | 79.4        |
---------------------------

Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/train/rl-video-episode-20.mp4


In [2]:
# Test the trained policy
test_output_folder = os.path.join(output_folder, 'test')
shutil.rmtree(test_output_folder, ignore_errors=True)
os.makedirs(test_output_folder, exist_ok=True)
env = RecordVideo(
    gym.make("Acrobot-v1", max_episode_steps=512, render_mode="rgb_array"),
    test_output_folder,
    episode_trigger=(lambda episode_id: True),
)
seed = 1
obs, info = env.reset(seed=seed)
T = 10000
for i in range(T):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    if done or truncated:
        if i + 1 == T:
            # will close outside the loop, no need to reset
            continue
        seed += 1
        obs, info = env.reset(seed=seed)
env.close()

/Users/jsukpk/Workstation/CSEM/cs395t-final-project/.env/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-0.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-0.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-1.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-1.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-2.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-2.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-3.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-3.mp4



...

Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-61.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-62.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-62.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-62.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-63.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-63.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-63.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-64.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-64.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-64.mp4
Moviepy - Building video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-65.mp4.
Moviepy - Writing video /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-65.mp4



Moviepy - Done !
Moviepy - video ready /Users/jsukpk/Workstation/CSEM/cs395t-final-project/gym-results/test/rl-video-episode-65.mp4
